In [1]:
import mysql.connector
import pandas as pd

# Connect to MySQL database
conn = mysql.connector.connect(
    host="localhost",
    port=3306,
    user="root",
    password="Sabghat@@@90",
    database="little_lemon"
)
cursor = conn.cursor()

In [21]:
# Create Orders Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Orders (
    OrderID VARCHAR(20) PRIMARY KEY,
    OrderDate DATE,
    ShipDate DATE,
    ShipMode VARCHAR(50),
    CustomerID VARCHAR(20),
    Country VARCHAR(50),
    City VARCHAR(50),
    State VARCHAR(50),
    PostalCode VARCHAR(20),
    Region VARCHAR(50)
)
''')

# Create Products Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Products (
    ProductID VARCHAR(20) PRIMARY KEY,
    Category VARCHAR(50),
    SubCategory VARCHAR(50),
    ProductName VARCHAR(255)
)
''')

# Create OrderDetails Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS OrderDetails (
    RowID INT AUTO_INCREMENT PRIMARY KEY,
    OrderID VARCHAR(20),  
    ProductID VARCHAR(20),
    Sales DECIMAL(10, 2),
    Quantity INT,
    Discount DECIMAL(5, 2),
    Profit DECIMAL(10, 2),
    FOREIGN KEY (OrderID) REFERENCES Orders(OrderID), 
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
)
''')

# Create Cart Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Cart (
    id INT PRIMARY KEY AUTO_INCREMENT,
    menuitem_id VARCHAR(20),
    quantity INT,
    FOREIGN KEY (menuitem_id) REFERENCES Products(ProductID)
)
''')

cursor.execute('''
CREATE TABLE Bookings (
    BookingID INT AUTO_INCREMENT PRIMARY KEY,
    CustomerID INT NOT NULL,
    TableNumber INT NOT NULL,
    BookingDate DATE NOT NULL,
    Status VARCHAR(20) NOT NULL
);
''')

# Commit the changes
conn.commit()

In [3]:
df = pd.read_csv("superstore.csv")

In [4]:
date_columns = ['Order Date', 'Ship Date']  # Add all date columns here
for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

In [5]:
df = df.drop_duplicates(subset=['Order ID'], keep='first')
df = df.drop_duplicates(subset=['Product ID'], keep='first')

In [6]:
if 'Order ID' in df.columns:
    for _, row in df.iterrows():
        query = '''
        INSERT INTO Orders (OrderID, OrderDate, ShipDate, ShipMode, CustomerID, Country, City, State, PostalCode, Region)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        '''
        values = (
            row['Order ID'], row['Order Date'], row['Ship Date'], row['Ship Mode'],
            row['Customer ID'], row['Country'], row['City'], row['State'],
            row['Postal Code'], row['Region']
        )
        cursor.execute(query, values)

In [7]:
if 'Product ID' in df.columns:
    for _, row in df.iterrows():
        query = '''
        INSERT INTO Products (ProductID, Category, SubCategory, ProductName)
        VALUES (%s, %s, %s, %s)
        '''
        values = (row['Product ID'], row['Category'], row['Sub-Category'], row['Product Name'])
        cursor.execute(query, values)

In [8]:
if 'Row ID' in df.columns:
    for _, row in df.iterrows():
        query = '''
        INSERT INTO OrderDetails (RowID, OrderID, ProductID, Sales, Quantity, Discount, Profit)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        '''
        values = (
            row['Row ID'], row['Order ID'], row['Product ID'], row['Sales'],
            row['Quantity'], row['Discount'], row['Profit']
        )
        cursor.execute(query, values)

In [9]:
conn.commit()

In [12]:
# Create the GetMaxQuantity procedure
get_max_quantity_procedure_query = """
CREATE PROCEDURE GetMaxQuantity()
BEGIN
    SELECT MAX(Quantity) AS MaxQuantity FROM OrderDetails;
END
"""

try:
    cursor.execute(get_max_quantity_procedure_query)
    print("Stored procedure GetMaxQuantity() created successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

Stored procedure GetMaxQuantity() created successfully.


In [13]:
add_booking_procedure = """
CREATE PROCEDURE AddBooking(
    IN customer_id INT,  
    IN table_number INT,  
    IN booking_date DATE,  
    IN status VARCHAR(20)
)
BEGIN
    INSERT INTO Bookings (CustomerID, TableNumber, BookingDate, Status)
    VALUES (customer_id, table_number, booking_date, status);
END
"""

try:
    cursor.execute(add_booking_procedure)
    print("Stored procedure for AddBooking() created successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

Stored procedure for AddBooking() created successfully.


In [19]:
update_booking_procedure_qurey = """
CREATE PROCEDURE UpdateBooking(
    IN booking_id INT,  
    IN table_number INT,  
    IN booking_date DATE,  
    IN status VARCHAR(20)
)
BEGIN
    UPDATE Bookings 
    SET TableNumber = table_number, BookingDate = booking_date, Status = status
    WHERE BookingID = booking_id;
END;
"""

try:
    cursor.execute(update_booking_procedure_qurey)
    print("Stored procedure for UpdateBooking() created successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

Stored procedure for UpdateBooking() created successfully.


In [15]:
cancel_booking_procedure_qurey = """
CREATE PROCEDURE CancelBooking(
    IN booking_id INT
)
BEGIN
    UPDATE Bookings 
    SET Status = 'Cancelled' 
    WHERE BookingID = booking_id;
END
"""

try:
    cursor.execute(cancel_booking_procedure_qurey)
    print("Stored procedure for CancelBooking() created successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

Stored procedure for CancelBooking() created successfully.


In [18]:
manage_booking_procedure_query = """
CREATE PROCEDURE ManageBooking(
    IN action_type VARCHAR(10),  -- 'ADD', 'UPDATE', 'CANCEL'
    IN booking_id INT,           -- Booking ID (required for UPDATE/CANCEL)
    IN customer_id INT,          -- Customer ID (for ADD)
    IN table_number INT,         -- Table number (for ADD/UPDATE)
    IN booking_date DATE,        -- Booking date (for ADD/UPDATE)
    IN status VARCHAR(20)        -- Status: 'Confirmed', 'Cancelled', etc.
)
BEGIN
    IF action_type = 'ADD' THEN
        INSERT INTO Bookings (CustomerID, TableNumber, BookingDate, Status)
        VALUES (customer_id, table_number, booking_date, status);

    ELSEIF action_type = 'UPDATE' THEN
        UPDATE Bookings
        SET TableNumber = table_number, BookingDate = booking_date, Status = status
        WHERE BookingID = booking_id;

    ELSEIF action_type = 'CANCEL' THEN
        UPDATE Bookings
        SET Status = 'Cancelled'
        WHERE BookingID = booking_id;
        END IF;
END
"""

try:
    cursor.execute(manage_booking_procedure_query)
    print("Stored procedure for ManageBooking() created successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

Stored procedure for ManageBooking() created successfully.


In [23]:
def test_add_booking():
    try:
        cursor.callproc("AddBooking", (1, 5, "2025-03-10", "Confirmed"))
        conn.commit()
        print("AddBooking() executed successfully.")
    except mysql.connector.Error as err:
        print(f"Error in AddBooking(): {err}")

def test_update_booking():
    try:
        cursor.callproc("UpdateBooking", (1, 3, "2025-03-15", "Rescheduled"))
        conn.commit()
        print("UpdateBooking() executed successfully.")
    except mysql.connector.Error as err:
        print(f"Error in UpdateBooking(): {err}")

def test_cancel_booking():
    try:
        cursor.callproc("CancelBooking", (1,))
        conn.commit()
        print("CancelBooking() executed successfully.")
    except mysql.connector.Error as err:
        print(f"Error in CancelBooking(): {err}")

def test_manage_booking():
    try:
        cursor.callproc("ManageBooking", ("ADD", None, 2, 4, "2025-03-20", "Confirmed"))
        conn.commit()
        print("ManageBooking(ADD) executed successfully.")

        cursor.callproc("ManageBooking", ("UPDATE", 2, None, 6, "2025-03-22", "Updated"))
        conn.commit()
        print("ManageBooking(UPDATE) executed successfully.")

        cursor.callproc("ManageBooking", ("CANCEL", 2, None, None, None, None))
        conn.commit()
        print("ManageBooking(CANCEL) executed successfully.")

    except mysql.connector.Error as err:
        print(f"Error in ManageBooking(): {err}")

test_add_booking()
test_update_booking()
test_cancel_booking()
test_manage_booking()


AddBooking() executed successfully.
UpdateBooking() executed successfully.
CancelBooking() executed successfully.
ManageBooking(ADD) executed successfully.
ManageBooking(UPDATE) executed successfully.
ManageBooking(CANCEL) executed successfully.


In [3]:
# Sample booking data
bookings_data = [
    (1, 3, '2025-03-10', 'Confirmed'),
    (2, 5, '2025-03-12', 'Pending'),
    (3, 2, '2025-03-15', 'Confirmed'),
    (4, 7, '2025-03-18', 'Cancelled'),
    (5, 4, '2025-03-20', 'Confirmed')
]

# Insert data into the Bookings table
insert_query = "INSERT INTO Bookings (CustomerID, TableNumber, BookingDate, Status) VALUES (%s, %s, %s, %s)"

try:
    cursor.executemany(insert_query, bookings_data)
    conn.commit()
    print("Sample bookings inserted successfully.")
except mysql.connector.Error as err:
    print(f"Error inserting bookings: {err}")

Sample bookings inserted successfully.
